In [1]:
import numpy as np
import os
import pandas as pd
import random
import torch
import torchaudio

from time import time
from torch import nn
from torch.utils.data import DataLoader

from msc_dataset import MSCDataset

# For data augmentation
import torch.nn.functional as F

In [2]:
# Device setup for Mac M4 Pro (MPS), CUDA (NVIDIA), or CPU fallback
if torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
elif torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print(f"Using device: {DEVICE}")

Using device: mps


In [3]:
# ==================== CONFIGURATION - STRATEGY 1 ====================
# Focus: Fine-tuning esistente con miglioramenti mirati
CFG = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.04,
    'frame_step_in_s': 0.02,
    'n_mels': 40,
    'f_min': 20,  # Ottimizzato per voce umana
    'f_max': 4000,
    'seed': 42,
    
    # Training Parameters - MODIFICATI
    'train_steps': 6000,  # +1000 steps per epoch (era 5000)
    'train_batch_size': 32,  # Ridotto per better convergence (era 64)
    'learning_rate': 0.0005,  # Ridotto per fine-tuning più preciso (era 0.001)
    'epochs': 60,  # Più epochs con patience (era 50)
    
    # Data Augmentation - RIDOTTA per evitare overfitting
    'time_shift_ms': 80,  # Ridotto (era 100)
    'noise_level': 0.003,  # Ridotto (era 0.005)
    'time_stretch_factor': 0.08,  # Ridotto (era 0.1)
    
    # Label Smoothing - NUOVO
    'label_smoothing': 0.1,  # Aiuta generalizzazione
}

# Define the set of target classes
CLASSES = ['stop', 'up']

# Set Deterministic Behaviour
torch.manual_seed(CFG['seed'])
np.random.seed(CFG['seed'])
random.seed(CFG['seed'])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [4]:
# ==================== MEL-SPECTROGRAM FEATURE EXTRACTOR ====================
class MelSpectrogramExtractor(nn.Module):
    """ONNX-compatible Mel-Spectrogram feature extractor"""
    def __init__(self):
        super().__init__()
        self.mel_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=CFG['sampling_rate'],
            n_fft=int(CFG['frame_length_in_s'] * CFG['sampling_rate']),
            hop_length=int(CFG['frame_step_in_s'] * CFG['sampling_rate']),
            n_mels=CFG['n_mels'],
            f_min=CFG['f_min'],
            f_max=CFG['f_max'],
            window_fn=torch.hann_window,
            power=2.0,
            normalized=False,
            center=True,
            pad_mode="reflect"
        )
        
    def forward(self, waveform):
        # waveform: (batch, samples)
        mel_spec = self.mel_transform(waveform)  # (batch, n_mels, time)
        
        # Log scale
        log_mel = torch.log(mel_spec + 1e-9)
        
        # Normalize to [-1, 1] range (per-sample)
        log_mel = (log_mel - log_mel.mean(dim=[1, 2], keepdim=True)) / (log_mel.std(dim=[1, 2], keepdim=True) + 1e-9)
        
        return log_mel.unsqueeze(1)  # (batch, 1, n_mels, time)

In [5]:
# ==================== DATA AUGMENTATION ====================
class AudioAugmentation:
    """Data augmentation per audio waveforms"""
    def __init__(self, config, training=True):
        self.config = config
        self.training = training
        
    def time_shift(self, waveform):
        """Shift audio randomly in time"""
        if not self.training or random.random() > 0.5:
            return waveform
            
        shift_samples = int(random.uniform(-self.config['time_shift_ms'], 
                                          self.config['time_shift_ms']) 
                          * self.config['sampling_rate'] / 1000)
        return torch.roll(waveform, shifts=shift_samples, dims=-1)
    
    def add_noise(self, waveform):
        """Add background white noise"""
        if not self.training or random.random() > 0.5:
            return waveform
            
        noise = torch.randn_like(waveform) * self.config['noise_level']
        return waveform + noise
    
    def time_stretch(self, waveform):
        """Slightly speed up or slow down audio"""
        if not self.training or random.random() > 0.5:
            return waveform
            
        rate = 1.0 + random.uniform(-self.config['time_stretch_factor'], 
                                    self.config['time_stretch_factor'])
        
        # Simple resampling-based time stretch
        stretched = F.interpolate(
            waveform.unsqueeze(0), 
            size=int(waveform.shape[-1] * rate),
            mode='linear',
            align_corners=False
        ).squeeze(0)
        
        # Pad or crop to original length
        target_len = waveform.shape[-1]
        if stretched.shape[-1] < target_len:
            stretched = F.pad(stretched, (0, target_len - stretched.shape[-1]))
        else:
            stretched = stretched[..., :target_len]
            
        return stretched
    
    def __call__(self, waveform):
        """Apply all augmentations in sequence"""
        waveform = self.time_shift(waveform)
        waveform = self.add_noise(waveform)
        waveform = self.time_stretch(waveform)
        return waveform

In [6]:
# ==================== CNN MODEL (IMPROVED) ====================
class KeywordSpotter(nn.Module):
    """Enhanced CNN for Up/Stop classification with Dropout"""
    def __init__(self, num_classes=2, dropout=0.3):
        super().__init__()
        
        # Block 1: 1 → 64 channels
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout2d(dropout * 0.5)  # Light dropout early
        
        # Block 2: 64 → 64 channels, downsample
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.dropout2 = nn.Dropout2d(dropout * 0.5)
        
        # Block 3: 64 → 128 channels, downsample
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=True)
        self.dropout3 = nn.Dropout2d(dropout)
        
        # Block 4: 128 → 128 channels
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU(inplace=True)
        
        # Block 5: 128 → 256 channels (NEW)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(256)
        self.relu5 = nn.ReLU(inplace=True)
        self.dropout5 = nn.Dropout2d(dropout)
        
        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # Classifier with dropout
        self.dropout_fc = nn.Dropout(dropout)
        self.fc = nn.Linear(256, num_classes, bias=True)
        
    def forward(self, x):
        # x: (batch, 1, 40, 49)
        x = self.dropout1(self.relu1(self.bn1(self.conv1(x))))  # (batch, 64, 40, 49)
        x = self.dropout2(self.relu2(self.bn2(self.conv2(x))))  # (batch, 64, 20, 25)
        x = self.dropout3(self.relu3(self.bn3(self.conv3(x))))  # (batch, 128, 10, 13)
        x = self.relu4(self.bn4(self.conv4(x)))                 # (batch, 128, 10, 13)
        x = self.dropout5(self.relu5(self.bn5(self.conv5(x))))  # (batch, 256, 10, 13)
        
        x = self.gap(x)  # (batch, 256, 1, 1)
        x = x.view(x.size(0), -1)  # (batch, 256)
        x = self.dropout_fc(x)
        x = self.fc(x)  # (batch, 2)
        
        return x

In [7]:
# ==================== MAIN TRAINING PIPELINE ====================
print("=" * 60)
print("UP/STOP KEYWORD SPOTTER - STRATEGY 1 (ENHANCED)")
print("=" * 60)
print(f"Device: {DEVICE}")
print(f"Mel-Spectrogram config: n_mels={CFG['n_mels']}, n_fft={int(CFG['frame_length_in_s'] * CFG['sampling_rate'])}, hop={int(CFG['frame_step_in_s'] * CFG['sampling_rate'])}")
print(f"Training config: epochs={CFG['epochs']}, batch_size={CFG['train_batch_size']}, lr={CFG['learning_rate']}")
print(f"Data Augmentation: time_shift=±{CFG['time_shift_ms']}ms, noise={CFG['noise_level']}, stretch=±{CFG['time_stretch_factor']*100}%")
print(f"Label Smoothing: {CFG['label_smoothing']}")
print("=" * 60)

# Create Mel-Spectrogram transform
transform = MelSpectrogramExtractor()

# Create augmentation
train_augmentation = AudioAugmentation(CFG, training=True)
val_augmentation = AudioAugmentation(CFG, training=False)

# Create datasets
print("\n📁 Loading datasets...")
train_dataset = MSCDataset(
    root='.',
    classes=CLASSES,
    split='training',
    preprocess=None,
)

val_dataset = MSCDataset(
    root='.',
    classes=CLASSES,
    split='validation',
    preprocess=None,
)

test_dataset = MSCDataset(
    root='.',
    classes=CLASSES,
    split='testing',
    preprocess=None,
)

# Create dataloaders with RandomSampler for training
sampler = torch.utils.data.RandomSampler(
    train_dataset,
    replacement=True,
    num_samples=CFG['train_steps'] * CFG['train_batch_size'],
)
train_loader = DataLoader(
    train_dataset,
    batch_size=CFG['train_batch_size'],
    sampler=sampler,
    num_workers=0,  # Set to 0 for macOS compatibility
)

val_loader = DataLoader(val_dataset, batch_size=100, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=100, num_workers=0)

# Initialize models
print("\n🏗️  Initializing models...")
feature_extractor = MelSpectrogramExtractor().to(DEVICE)
model = KeywordSpotter(num_classes=len(CLASSES), dropout=0.3).to(DEVICE)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {total_params:,}")
print(f"Estimated size (float32): {total_params * 4 / 1024:.2f} KB")
print(f"Estimated size (int8): {total_params / 1024:.2f} KB")

# Loss with Label Smoothing and optimizer - STRATEGY 1
loss_module = nn.CrossEntropyLoss(label_smoothing=CFG['label_smoothing'])
optimizer = torch.optim.AdamW(  # AdamW invece di Adam
    model.parameters(), 
    lr=CFG['learning_rate'], 
    weight_decay=2e-4,  # Aumentato (era 1e-4)
    betas=(0.9, 0.999)
)

# Scheduler - CosineAnnealingWarmRestarts per cicli di learning rate
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, 
    T_0=10,  # Restart ogni 10 epochs
    T_mult=2,
    eta_min=1e-7
)

# Early stopping setup
best_val_acc = 0
best_val_loss = float('inf')
patience = 20  # Più patience (era 15)
patience_counter = 0
best_model_state = None


UP/STOP KEYWORD SPOTTER - STRATEGY 1 (ENHANCED)
Device: mps
Mel-Spectrogram config: n_mels=40, n_fft=640, hop=320
Training config: epochs=60, batch_size=32, lr=0.0005
Data Augmentation: time_shift=±80ms, noise=0.003, stretch=±8.0%
Label Smoothing: 0.1

📁 Loading datasets...
Using data folder: ./msc-training
Loaded 1600 samples from ./msc-training for classes ['stop', 'up']
Using data folder: ./msc-validation
Loaded 200 samples from ./msc-validation for classes ['stop', 'up']
Using data folder: ./msc-testing
Loaded 200 samples from ./msc-testing for classes ['stop', 'up']

🏗️  Initializing models...
Model parameters: 555,330
Estimated size (float32): 2169.26 KB
Estimated size (int8): 542.31 KB


In [8]:
# ==================== TRAINING & EVALUATION FUNCTIONS ====================

def evaluate(model, feature_extractor, loader, device):
    """Evaluate model on validation/test set"""
    model.eval()
    correct = total = 0
    total_loss = 0
    loss_module = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for batch in loader:
            x = batch['x'].squeeze(1).to(device)
            y = batch['y'].to(device)
            features = feature_extractor(x)
            output = model(features)
            predictions = output.argmax(dim=1)
            correct += (predictions == y).sum().item()
            total += y.size(0)
            total_loss += loss_module(output, y).item()
    
    accuracy = (correct / total) * 100
    avg_loss = total_loss / len(loader)
    return accuracy, avg_loss


def train_epoch(model, feature_extractor, train_loader, optimizer, loss_module, device, 
                steps_per_epoch, current_epoch, augmentation):
    """Train model for one epoch with data augmentation"""
    model.train()
    
    start_step = current_epoch * steps_per_epoch
    end_step = start_step + steps_per_epoch
    
    epoch_loss = 0
    step_count = 0
    
    for step, batch in enumerate(train_loader):
        if step < start_step:
            continue
        if step >= end_step:
            break
            
        x = batch['x'].squeeze(1)  # Keep on CPU for augmentation
        y = batch['y'].to(device)
        
        # Apply data augmentation on CPU
        x_augmented = []
        for i in range(x.shape[0]):
            aug_sample = augmentation(x[i:i+1])
            x_augmented.append(aug_sample)
        x = torch.cat(x_augmented, dim=0).to(device)
        
        # Extract features
        with torch.no_grad():
            features = feature_extractor(x)
        
        # Forward pass
        output = model(features)
        loss = loss_module(output, y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        step_count += 1
        
        if ((step + 1) % 100) == 0 or step == 0:
            print(f'Step={step}; Training Loss={loss.item():.4f}')
    
    avg_epoch_loss = epoch_loss / step_count if step_count > 0 else 0
    return avg_epoch_loss


def test_model(model, feature_extractor, test_loader, device):
    """Test model on test dataset"""
    model.eval()
    correct = total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            x = batch['x'].squeeze(1).to(device)
            y = batch['y'].to(device)
            
            # Extract features
            features = feature_extractor(x)
            
            output = model(features)
            predictions = output.argmax(dim=1)
            
            correct += (predictions == y).sum().item()
            total += y.size(0)
    
    accuracy = (correct / total) * 100
    return accuracy

In [9]:
# ==================== TRAINING LOOP - STRATEGY 1 ====================
print("\n🚀 Starting training (Strategy 1)...")

steps_per_epoch = len(train_loader) // CFG['epochs']
current_epoch = 0

train_history = {'epoch': [], 'train_loss': [], 'val_acc': [], 'val_loss': [], 'lr': []}

for epoch in range(CFG['epochs']):
    print(f"\n{'='*60}")
    print(f"EPOCH {epoch+1}/{CFG['epochs']}")
    print(f"{'='*60}")
    
    # Train for one epoch
    train_loss = train_epoch(
        model, feature_extractor, train_loader, optimizer, loss_module, 
        DEVICE, steps_per_epoch, epoch, train_augmentation
    )
    
    current_epoch += 1
    
    # Evaluate on validation set
    val_acc, val_loss = evaluate(model, feature_extractor, val_loader, DEVICE)
    
    # Get current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f'\n📊 Epoch {current_epoch} Summary:')
    print(f'   Train Loss: {train_loss:.4f}')
    print(f'   Val Acc: {val_acc:.2f}%')
    print(f'   Val Loss: {val_loss:.4f}')
    print(f'   Learning Rate: {current_lr:.6f}')
    
    # Save history
    train_history['epoch'].append(current_epoch)
    train_history['train_loss'].append(train_loss)
    train_history['val_acc'].append(val_acc)
    train_history['val_loss'].append(val_loss)
    train_history['lr'].append(current_lr)
    
    # Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
        patience_counter = 0
        print(f'✅ New best model! Val Acc={val_acc:.2f}%')
    else:
        patience_counter += 1
        print(f'⏳ No improvement. Patience: {patience_counter}/{patience}')
    
    if patience_counter >= patience:
        print(f'\n🛑 Early stopping at epoch {current_epoch}')
        break
    
    # CosineAnnealingWarmRestarts step (chiamato dopo ogni epoch)
    scheduler.step()

# Load best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f'\n✅ Loaded best model with Val Acc={best_val_acc:.2f}%')

print(f"\n{'='*60}")
print(f"Training completed after {current_epoch} epochs")
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print(f"{'='*60}")



🚀 Starting training (Strategy 1)...

EPOCH 1/60
Step=0; Training Loss=0.7232
Step=0; Training Loss=0.7232
Step=99; Training Loss=0.5617

📊 Epoch 1 Summary:
   Train Loss: 0.6593
   Val Acc: 84.50%
   Val Loss: 0.4545
   Learning Rate: 0.000500
✅ New best model! Val Acc=84.50%

EPOCH 2/60
Step=99; Training Loss=0.5617

📊 Epoch 1 Summary:
   Train Loss: 0.6593
   Val Acc: 84.50%
   Val Loss: 0.4545
   Learning Rate: 0.000500
✅ New best model! Val Acc=84.50%

EPOCH 2/60
Step=199; Training Loss=0.4531

📊 Epoch 2 Summary:
   Train Loss: 0.5477
   Val Acc: 90.50%
   Val Loss: 0.3107
   Learning Rate: 0.000488
✅ New best model! Val Acc=90.50%

EPOCH 3/60
Step=199; Training Loss=0.4531

📊 Epoch 2 Summary:
   Train Loss: 0.5477
   Val Acc: 90.50%
   Val Loss: 0.3107
   Learning Rate: 0.000488
✅ New best model! Val Acc=90.50%

EPOCH 3/60
Step=299; Training Loss=0.4421

📊 Epoch 3 Summary:
   Train Loss: 0.4824
   Val Acc: 87.00%
   Val Loss: 0.3148
   Learning Rate: 0.000452
⏳ No improvement. Pa

In [10]:
# ==================== TEST EVALUATION ====================
print("\n📊 Evaluating model on test set...")

test_accuracy = test_model(model, feature_extractor, test_loader, DEVICE)
print(f'\n🎯 Test Accuracy: {test_accuracy:.2f}%')

if test_accuracy > 99.4:
    print("✅ PASSED: Accuracy > 99.4%")
else:
    print("❌ FAILED: Accuracy <= 99.4%")



📊 Evaluating model on test set...

🎯 Test Accuracy: 99.00%
❌ FAILED: Accuracy <= 99.4%


In [11]:
# ==================== SAVE MODEL ====================
print("\n" + "="*60)
print("SAVING MODEL")
print("="*60)

timestamp = int(time())
saved_model_dir = './saved_models/'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)

print(f'Model Timestamp: {timestamp}')

model.eval()
feature_extractor.eval()

# Move models to CPU for ONNX export (MPS not supported for export)
print("\n🔄 Moving models to CPU for ONNX export...")
model_cpu = model.cpu()
feature_extractor_cpu = feature_extractor.cpu()

# Export Feature Extractor to ONNX
print("\n📦 Exporting Feature Extractor to ONNX...")
torch.onnx.export(
    feature_extractor_cpu,  # model to export
    torch.randn(1, 16000),  # inputs of the model (waveform)
    f'{saved_model_dir}/{timestamp}_frontend.onnx',  # filename of the ONNX model
    input_names=['input'],  # input name in the ONNX model
    dynamo=True,
    optimize=True,
    report=False,
    external_data=False,
)
print(f"✅ Feature extractor saved: {saved_model_dir}/{timestamp}_frontend.onnx")

# Export Model to ONNX
print("\n📦 Exporting Model to ONNX...")
# Get a sample waveform from training dataset and extract features
sample_waveform = train_dataset[0]['x'].squeeze(0).unsqueeze(0).cpu()  # (1, 16000)
sample_features = feature_extractor_cpu(sample_waveform)  # (1, 1, n_mels, time)
torch.onnx.export(
    model_cpu,  # model to export
    sample_features,  # inputs of the model (mel-spectrogram features)
    f'{saved_model_dir}/{timestamp}_model.onnx',  # filename of the ONNX model
    input_names=['input'],  # input name in the ONNX model
    dynamo=True,
    optimize=True,
    report=False,
    external_data=False,
)
print(f"✅ Model saved: {saved_model_dir}/{timestamp}_model.onnx")

# Check sizes
fe_size = os.path.getsize(f'{saved_model_dir}/{timestamp}_frontend.onnx') / 1024
model_size = os.path.getsize(f'{saved_model_dir}/{timestamp}_model.onnx') / 1024
total_size = fe_size + model_size

print("\n" + "="*60)
print("SIZE REPORT (ONNX - Float32)")
print("="*60)
print(f"Feature Extractor: {fe_size:.2f} KB")
print(f"Model: {model_size:.2f} KB")
print(f"Total: {total_size:.2f} KB")

if total_size < 300:
    print("✅ PASSED: Total size < 300 KB (before quantization)")
else:
    print("⚠️  WARNING: Size > 300 KB - quantization required!")

# Save Hyperparameters & Results
print("\n📝 Saving hyperparameters and results...")
output_dict = {
    'timestamp': timestamp,
    **CFG,
    'test_accuracy': test_accuracy
}

df = pd.DataFrame([output_dict])
output_path = './keyword_spotter_results.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)
print(f"✅ Results saved to {output_path}")

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)



SAVING MODEL
Model Timestamp: 1764415450

🔄 Moving models to CPU for ONNX export...

📦 Exporting Feature Extractor to ONNX...


W1129 12:24:10.481000 38428 torch/onnx/_internal/exporter/_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `MelSpectrogramExtractor([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `MelSpectrogramExtractor([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


W1129 12:24:10.965000 38428 torch/onnx/_internal/exporter/_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 4 of general pattern rewrite rules.
✅ Feature extractor saved: ./saved_models//1764415450_frontend.onnx

📦 Exporting Model to ONNX...
[torch.onnx] Obtain model graph for `KeywordSpotter([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `KeywordSpotter([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 10 of general pattern rewrite rules.
✅ Model saved: ./saved_models//1764415450_model.onnx

SIZE REPORT (ONNX - Float32)
Feature Extractor: 332.49 KB
Model: 2178.31 KB
Total: 2510.80 KB
⚠️  WARNING: Size > 300 KB - quantization required!

📝 Saving hyperparameters and results...
✅ Results saved to ./keyword_spotter_results.csv

TRAINING C